In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Fibrosis"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
161: 1158
43: 1158
126: 1158
234: 1158
83: 1158
114: 1158
10: 1158
62: 1158
57: 1158
7: 1158
25: 1158
1158: 1158
113: 1158
4: 1158
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
1141/1158 [============================>.] - ETA: 0s - loss: 5.0685 - leftLayer1_loss: 0.1154 - midLayer1_loss: 1.3741 - rightLayer1_loss: 1.1771 - leftLayer2_loss: 0.0900 - midLayer2_loss: 1.4374 - rightLayer2_loss: 0.8745
Epoch 00006: val_loss improved from 5.19663 to 5.06580, saving model to ./experiments/0.009_weights.h5
1158/1158 [==============================] - 3s 3ms/step - loss: 5.0694 - leftLayer1_loss: 0.1154 - midLayer1_loss: 1.3747 - rightLayer1_loss: 1.1769 - leftLayer2_loss: 0.0900 - midLayer2_loss: 1.4373 - rightLayer2_loss: 0.8751 - val_loss: 5.0658 - val_leftLayer1_loss: 0.1139 - val_midLayer1_loss: 1.3478 - val_rightLayer1_loss: 1.1186 - val_leftLayer2_loss: 0.0976 - val_midLayer2_loss: 1.3434 - val_rightLayer2_loss: 1.0445
Epoch 7/11
1140/1158 [============================>.] - ETA: 0s - loss: 4.9630 - leftLayer1_loss: 0.1127 - midLayer1_loss: 1.3726 - rightLayer1_loss: 1.1130 - leftLayer2_loss: 0.0852 - midLayer2_loss: 1.4348 - rightLayer2_loss: 0.8447


  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1158 steps, validate for 166 steps
Epoch 1/11
1145/1158 [============================>.] - ETA: 0s - loss: 6.3521 - leftLayer1_loss: 0.1329 - midLayer1_loss: 1.2838 - rightLayer1_loss: 1.7865 - leftLayer2_loss: 0.1228 - midLayer2_loss: 1.3742 - rightLayer2_loss: 1.6518
Epoch 00001: val_loss improved from inf to 6.10411, saving model to ./experiments/0.009999999999999998_weights.h5
1158/1158 [==============================] - 4s 3ms/step - loss: 6.3493 - leftLayer1_loss: 0.1329 - midLayer1_loss: 1.2840 - rightLayer1_loss: 1.7856 - leftLayer2_loss: 0.1228 - midLayer2_loss: 1.3744 - rightLayer2_loss: 1.6497 - val_loss: 6.1041 - val_leftLayer1_loss: 0.1313 - val_midLayer1_loss: 1.2677 - val_rightLayer1_loss: 1.6975 - val_leftLayer2_loss: 0.1168 - val_midLayer2_loss: 1.2983 - val_rightLayer2_loss: 1.5

1146/1158 [============================>.] - ETA: 0s - loss: 4.5719 - leftLayer1_loss: 0.1038 - midLayer1_loss: 1.2835 - rightLayer1_loss: 0.9522 - leftLayer2_loss: 0.0705 - midLayer2_loss: 1.3770 - rightLayer2_loss: 0.7849
Epoch 00011: val_loss improved from 4.62570 to 4.57383, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Fibrosis: 0.6160819264598354

auprc 0Fibrosis: 0.022220813466176076

auroc 1Fibrosis: 0.479039518620344

auprc 1Fibrosis: 0.015231086117729494

auroc 2Fibrosis: 0.5423272357341988

auprc 2Fibrosis: 0.021150477843869833

auroc 3Fibrosis: 0.40880761259931847

auprc 3Fibrosis: 0.015004306850952226

auroc 4Fibrosis: 0.27316337956033904

auprc 4Fibrosis: 0.009955885973820688

auroc 5Fibrosis: 0.41071750861296197

auprc 5Fibrosis: 0.015682087655406813

mean auroc: 0.4550228635978329

mean auprc: 0.01654077631799252

max auroc: 0

Epoch 4/11
1143/1158 [============================>.] - ETA: 0s - loss: 5.3102 - leftLayer1_loss: 0.1117 - midLayer1_loss: 1.3462 - rightLayer1_loss: 1.3530 - leftLayer2_loss: 0.0973 - midLayer2_loss: 1.4291 - rightLayer2_loss: 0.9729
Epoch 00004: val_loss improved from 5.51010 to 5.30268, saving model to ./experiments/0.010999999999999998_weights.h5
1158/1158 [==============================] - 3s 3ms/step - loss: 5.3095 - leftLayer1_loss: 0.1116 - midLayer1_loss: 1.3469 - rightLayer1_loss: 1.3524 - leftLayer2_loss: 0.0973 - midLayer2_loss: 1.4284 - rightLayer2_loss: 0.9729 - val_loss: 5.3027 - val_leftLayer1_loss: 0.1103 - val_midLayer1_loss: 1.3236 - val_rightLayer1_loss: 1.2837 - val_leftLayer2_loss: 0.1021 - val_midLayer2_loss: 1.3542 - val_rightLayer2_loss: 1.1289
Epoch 5/11
1155/1158 [============================>.] - ETA: 0s - loss: 5.1376 - leftLayer1_loss: 0.1091 - midLayer1_loss: 1.3491 - rightLayer1_loss: 1.2571 - leftLayer2_loss: 0.0913 - midLayer2_loss: 1.4265 - rightLayer

...

1135/1158 [============================>.] - ETA: 0s - loss: 4.7122 - leftLayer1_loss: 0.0984 - midLayer1_loss: 1.3540 - rightLayer1_loss: 0.9345 - leftLayer2_loss: 0.0741 - midLayer2_loss: 1.4603 - rightLayer2_loss: 0.7908
Epoch 00011: val_loss improved from 4.77230 to 4.72045, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Fibrosis: 0.7085834991092284

auprc 0Fibrosis: 0.056145690244525086

auroc 1Fibrosis: 0.5408304965566926

auprc 1Fibrosis: 0.017179383678749455

auroc 2Fibrosis: 0.5599420979656062

auprc 2Fibrosis: 0.023573881441725845

auroc 3Fibrosis: 0.5761709886050819

auprc 3Fibrosis: 0.020977993484819588

auroc 4Fibrosis: 0.4960114907915214

auprc 4Fibrosis: 0.014461691140439244

auroc 5Fibrosis: 0.3434168132929113

auprc 5Fibrosis: 0.013768694266540177

mean auroc: 0.5374925643868403

mean auprc: 0.02435122237613323

max auroc: 0.7

Epoch 4/11
1137/1158 [============================>.] - ETA: 0s - loss: 5.4081 - leftLayer1_loss: 0.1149 - midLayer1_loss: 1.3415 - rightLayer1_loss: 1.3928 - leftLayer2_loss: 0.1040 - midLayer2_loss: 1.4700 - rightLayer2_loss: 0.9849
Epoch 00004: val_loss improved from 5.61978 to 5.40427, saving model to ./experiments/0.012999999999999996_weights.h5
1158/1158 [==============================] - 3s 2ms/step - loss: 5.4100 - leftLayer1_loss: 0.1148 - midLayer1_loss: 1.3422 - rightLayer1_loss: 1.3922 - leftLayer2_loss: 0.1040 - midLayer2_loss: 1.4714 - rightLayer2_loss: 0.9853 - val_loss: 5.4043 - val_leftLayer1_loss: 0.1138 - val_midLayer1_loss: 1.3153 - val_rightLayer1_loss: 1.3303 - val_leftLayer2_loss: 0.1092 - val_midLayer2_loss: 1.3605 - val_rightLayer2_loss: 1.1753
Epoch 5/11
1141/1158 [============================>.] - ETA: 0s - loss: 5.2175 - leftLayer1_loss: 0.1123 - midLayer1_loss: 1.3421 - rightLayer1_loss: 1.2946 - leftLayer2_loss: 0.0983 - midLayer2_loss: 1.4530 - rightLayer

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1158 steps, validate for 166 steps
Epoch 1/11
1146/1158 [============================>.] - ETA: 0s - loss: 6.5098 - leftLayer1_loss: 0.1242 - midLayer1_loss: 1.4223 - rightLayer1_loss: 1.7561 - leftLayer2_loss: 0.1272 - midLayer2_loss: 1.3879 - rightLayer2_loss: 1.6922
Epoch 00001: val_loss improved from inf to 6.16350, saving model to ./experiments/0.013999999999999995_weights.h5
1158/1158 [==============================] - 4s 3ms/step - loss: 6.5070 - leftLayer1_loss: 0.1242 - midLayer1_loss: 1.4224 - rightLayer1_loss: 1.7552 - leftLayer2_loss: 0.1272 - midLayer2_loss: 1.3882 - rightLayer2_loss: 1.6899 - val_loss: 6.1635 - val_leftLayer1_loss: 0.1225 - val_midLayer1_loss: 1.3961 - val_rightLayer1_loss: 1.6546 - val_leftLayer2_loss: 0.1219 - val_midLayer2_loss: 1.2823 - val_rightLayer2_loss: 1.5

1144/1158 [============================>.] - ETA: 0s - loss: 4.6825 - leftLayer1_loss: 0.0950 - midLayer1_loss: 1.4220 - rightLayer1_loss: 0.9187 - leftLayer2_loss: 0.0702 - midLayer2_loss: 1.3944 - rightLayer2_loss: 0.7822
Epoch 00011: val_loss improved from 4.66035 to 4.61237, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Fibrosis: 0.43053733418342766

auprc 0Fibrosis: 0.013040549056484378

auroc 1Fibrosis: 0.4227356664866849

auprc 1Fibrosis: 0.012962322014687697

auroc 2Fibrosis: 0.6085871538137468

auprc 2Fibrosis: 0.03268912264485903

auroc 3Fibrosis: 0.4477541840734486

auprc 3Fibrosis: 0.016342355047122087

auroc 4Fibrosis: 0.5459820028331469

auprc 4Fibrosis: 0.018041530099025385

auroc 5Fibrosis: 0.4609620609129102

auprc 5Fibrosis: 0.017267619753948547

mean auroc: 0.4860930670505608

mean auprc: 0.018390583102687857

max auroc: 0.

Epoch 4/11
1151/1158 [============================>.] - ETA: 0s - loss: 5.4002 - leftLayer1_loss: 0.1144 - midLayer1_loss: 1.4104 - rightLayer1_loss: 1.3520 - leftLayer2_loss: 0.1060 - midLayer2_loss: 1.4145 - rightLayer2_loss: 1.0030
Epoch 00004: val_loss improved from 5.64054 to 5.42836, saving model to ./experiments/0.014999999999999994_weights.h5
1158/1158 [==============================] - 3s 2ms/step - loss: 5.4025 - leftLayer1_loss: 0.1144 - midLayer1_loss: 1.4107 - rightLayer1_loss: 1.3522 - leftLayer2_loss: 0.1060 - midLayer2_loss: 1.4153 - rightLayer2_loss: 1.0039 - val_loss: 5.4284 - val_leftLayer1_loss: 0.1127 - val_midLayer1_loss: 1.3882 - val_rightLayer1_loss: 1.2840 - val_leftLayer2_loss: 0.1085 - val_midLayer2_loss: 1.3507 - val_rightLayer2_loss: 1.1843
Epoch 5/11
1150/1158 [============================>.] - ETA: 0s - loss: 5.2165 - leftLayer1_loss: 0.1117 - midLayer1_loss: 1.4093 - rightLayer1_loss: 1.2555 - leftLayer2_loss: 0.0995 - midLayer2_loss: 1.4116 - rightLayer

In [17]:
print(np.max(maxi))

0.7085834991092284
